In [1]:
# Jupyter notebook implementing the full workflow to generate strain maps of inputted micrograph images using the
# parent-child algorithm.
# Author: Sanket Gadgil, Date: 16/11/2020
# With guidance from: https://github.com/pyxem/pyxem-demos/blob/master/05%20Simulate%20Data%20-%20Strain%20Mapping.ipynb
%matplotlib qt
import hyperspy.api as hs
import numpy as np
from numpy.matlib import repmat
import matplotlib.pyplot as plt
import pyxem as pxm
import diffpy.structure
from pyxem.utils.sim_utils import sim_as_signal
from copy import deepcopy


def pixel_binning(raw_sig, root_of_nbin):
    '''
    Produce a binned image from an inputted signal micrograph image.
    
    Arguments:
    raw_sig -- input signal image data in the form of a Signal2D instance.
    root_of_nbin -- one dimension of the tile that will be averaged, 
                    eg. if root_of_nbin = 2 and the image dimensions are 256x256 
                    then the image will be split into four 128x128 pixel tiles and 
                    each tile will be averaged to produce a pixel in the binned image, 
                    which will be of dimensions: 2x2 pixels.

    Output:
    binned_result -- binned signal, also a Signal2D instance. Note that each of the 
                     new pixels also has a diffraction pattern which is an average of 
                     the diffraction patterns of the pixels in the binning tiles.
    '''
    pixels_binned = int(raw_sig.data.shape[0]/root_of_nbin)
    binned_result = raw_sig.rebin(
        scale=[pixels_binned, pixels_binned, 1, 1]
    )/(pixels_binned**2)  # Normalisation, since pixel binning accumulates without taking the average

    return binned_result

def log_sig(sig):
    '''
    Produce a log-scale diffraction patterns for each pixel in an input signal.
    
    Arguments:
    sig -- input signal in the form of a Signal2D instance.
    
    Output:
    logged_sig -- resulting signal with logged diffraction patterns, 
                  also a Signal2D instance.
    '''
    
    logged_sig = sig  # new variable to prevent overwriting sig
    for inav_x in range(logged_sig.data.shape[0]):
        for inav_y in range(logged_sig.data.shape[1]):
            # The next three lines raise the negative values in the signal to be equal to the 
            # smallest positive value, this ensures a correct result when taking the log of 
            # the signal data.
            sig_min_indices = logged_sig.data[inav_x, inav_y] <= 0
            min_thresh = np.min(logged_sig.data[inav_x, inav_y][logged_sig.data[inav_x, inav_y] > 0])
            logged_sig.data[inav_x, inav_y][sig_min_indices] = min_thresh
            
            logged_sig.data[inav_x, inav_y] = np.log10(logged_sig.data[inav_x, inav_y])
    
    return logged_sig

# Data import (change the path to point to the .xml file in question)
sig = hs.load("./acquisition_6/acquisition_6.xml")

# Log-scale signal
sig = log_sig(sig)


Please use sidpy.viz.plot_utils instead of pyUSID.viz.plot_utils. pyUSID.plot_utils will be removed in a future release of pyUSID


In [2]:
def binning(sig, scale, firstbin=False):
    '''
    Produce binned data and additional metadata.
    
    Arguments:
    sig -- input signal, instance of Signal2D.
    scale -- binning scale, eg. scale=2 will result in a 2x2 pixel image.
    firstbin -- flag signalling whether this is the first binning operation 
                in the parent-child algorithm.
    
    Output:
    binned_data -- binned signal, instance of Signal2D.
    binned_nx, binned_ny -- dimensions of binned_data.
    ref_nx, ref_ny -- dimensions of the binned_data at the previous level(parent).
                      If firstbin=True these are (1,1).
    '''
    binned_data = pixel_binning(sig, scale)
    binned_nx, binned_ny = binned_data.data.shape[0:2]
    
    # This returns either the dimensions of the reference data or the dimensions of 
    # the binned image in the previous level of the parent-child algorithm
    if not firstbin:
        ref_nx, ref_ny = int(binned_nx/2), int(binned_ny/2)
    else:
        ref_nx, ref_ny = 1, 1

    return [
        binned_data,
        binned_nx, binned_ny,
        ref_nx, ref_ny  # This is needed for model_create()
    ]


In [3]:
from pyxem.components.scalable_reference_pattern import ScalableReferencePattern

def model_create(
    binning_results,
    reference_data,
    first_model=False,
    fit_results=None
):
    '''
    Create a hyperspy model2D object from inputted binned signal data and 
    attach a pyxem ScalableReferencePattern which will be used in latter fitting.
    
    Arguments:
    binning_results -- output from binning().
    reference_data -- a reference signal2D object which provides a de facto zero-strain 
                      diffraction pattern which will be used in latter fitting.
    first_model -- flag indicating whether the function call is the first of the 
                   parent child algorithm.
    fit_results -- object of type numpy.ndarray which holds the results of the fitting 
                   of a previous level(parent) in the parent-child algorithm.
    
    Output:
    model -- model2D object which will be used in model_fit()
    '''
    
    # Unpack arguments
    binned_data = binning_results[0]
    binned_nx, binned_ny = binning_results[1], binning_results[2]
    ref_nx, ref_ny = binning_results[3], binning_results[4] 

    # Create model and add ScalableReferencePattern
    model = binned_data.create_model()
    ref_pattern0 = ScalableReferencePattern(reference_data.inav[0, 0])
    model.append(ref_pattern0)          
    
    # Regardless of bounded or unbounded fit, add limits to each parameter
    # (they'll only be relevant if bounded=True flag is passed to model_fit())
    for component in model:
        for param in component.free_parameters:
            if param.name == "d11":
                param.bmin, param.bmax = 0.9, 1.1
            elif param.name == "d12":
                param.bmin, param.bmax = -0.1, 0.1
            elif param.name == "d21":
                param.bmin, param.bmax = -0.1, 0.1
            elif param.name == "d22":
                param.bmin, param.bmax = 0.9, 1.1
            elif param.name == "t1":
                param.bmin, param.bmax = -0.01, 0.01
            elif param.name == "t2":
                param.bmin, param.bmax = -0.01, 0.01
    
    # inherit parameter data from the previous level of the parent-child algorithm
    if not first_model:
        d11_data = fit_results[0]
        d12_data = fit_results[1]
        d21_data = fit_results[2]
        d22_data = fit_results[3]
        t1_data = fit_results[4]
        t2_data = fit_results[5]

        # Map the inherited data to new expanded grid eg. (2x2)->(4x4)
        for ix in range(ref_nx):
            for iy in range(ref_ny):
                map_start_ix, map_end_ix = [ix*2, (ix+1)*2]
                map_start_iy, map_end_iy = [iy*2, (iy+1)*2]

                # The repmat function turns a parameter set at each grid point into a 
                # repeated (2x2) cluster so as to serve as an initial guess for the next level
                # of the parent-child algorithm.
                model[0].d11.map[
                    map_start_ix:map_end_ix,
                    map_start_iy:map_end_iy
                ] = repmat(np.array([d11_data[ix, iy]]), 2, 2)
                model[0].d12.map[
                    map_start_ix:map_end_ix,
                    map_start_iy:map_end_iy
                ] = repmat(np.array([d12_data[ix, iy]]), 2, 2)
                model[0].d21.map[
                    map_start_ix:map_end_ix,
                    map_start_iy:map_end_iy
                ] = repmat(np.array([d21_data[ix, iy]]), 2, 2)
                model[0].d22.map[
                    map_start_ix:map_end_ix,
                    map_start_iy:map_end_iy
                ] = repmat(np.array([d22_data[ix, iy]]), 2, 2)
                model[0].t1.map[
                    map_start_ix:map_end_ix,
                    map_start_iy:map_end_iy
                ] = repmat(np.array([t1_data[ix, iy]]), 2, 2)
                model[0].t2.map[
                    map_start_ix:map_end_ix,
                    map_start_iy:map_end_iy
                ] = repmat(np.array([t2_data[ix, iy]]), 2, 2)

    # return a list to allow other outputs to be included if necessary
    return [
        model,
    ]

In [4]:
# Please ensure parent_child_fit_library.py is either appropriately linked or in the same folder as this notebook.
from parent_child_fit_library import multifit

def model_fit(
    creation_results,
    first_fit=False,
    bounded=False
):
    '''
    Fit parameters of the model and return the parameters
    
    Arguments:
    creation results -- output from model_create()
    first_fit -- flag indicating whether the function call is the first of the 
                 parent child algorithm. 
    
    Output:
    d11_data, d12_data, ... -- Updated parameters.
    '''
    # Unpack arguments
    model = creation_results[0]
    
    # For independent(linear) fitting the itertype is not relevant but 
    # is included for consistency, compatibility and comparability.
    itertype = "flyback"
    multifit(model, iterpath=itertype, firstfit=first_fit, bounded=bounded)

    # Make copies of the fitted parameter sets to pass to model_create() in the next level
    # of the parent-child algorithm
    d11_data = deepcopy(model[0].d11.map)
    d12_data = deepcopy(model[0].d12.map)
    d21_data = deepcopy(model[0].d21.map)
    d22_data = deepcopy(model[0].d22.map)
    t1_data = deepcopy(model[0].t1.map)
    t2_data = deepcopy(model[0].t2.map)

    return [
        d11_data,
        d12_data,
        d21_data,
        d22_data,
        t1_data,
        t2_data
    ]


In [5]:
def reference_gen(synthetic=False):
    '''
    Generate reference image.
    
    Arguments:
    synthetic -- Flag indicating whether to use the synthetic image generated by 
                 synthetic_reference_image.py or use binning instead.
    
    Output:
    refernce_data -- Reference image as a Signal2D instance.
    '''
    reference_data = 0

    if synthetic:
        # Load saved data (change path to the location of the saved data in question)
        reference_data = np.loadtxt("ref_patt_synth.csv", delimiter=',')
        
        # Define metadata to be used when defining the dictionary for the Signal2D instance
        ref_dict0 = {'size':1, 'name':'scan_y', 'offset':36, 'scale':72, 'units':'nm'}
        ref_dict1 = {'size':1, 'name':'scan_x', 'offset':36, 'scale':72, 'units':'nm'}
        ref_dict2 = {'name':'width', 'offset':-0.19, 'scale':0.0029, 'units':'1/nm'}
        ref_dict3 = {'name':'height', 'offset':-0.19, 'scale':0.0029, 'units':'1/nm'}

        reference_data = hs.signals.Signal2D(np.array([[reference_data]]))

        # Enter metadata
        for dict_key in ref_dict0.keys():
            reference_data.axes_manager[0].__setattr__(dict_key, ref_dict0.get(dict_key))
        for dict_key in ref_dict1.keys():
            reference_data.axes_manager[1].__setattr__(dict_key, ref_dict1.get(dict_key))
        for dict_key in ref_dict2.keys():
            reference_data.axes_manager[2].__setattr__(dict_key, ref_dict2.get(dict_key))
        for dict_key in ref_dict3.keys():
            reference_data.axes_manager[3].__setattr__(dict_key, ref_dict3.get(dict_key))

        # Diagnostics (comment out if unnecessary)
        reference_data.plot(cmap='jet')
    else:
        # Normal reference data generated using binning.
        reference_data = pixel_binning(sig, 1)
        reference_data.plot(cmap="jet")
    
    return reference_data

In [8]:
# First level of the parent-child algorithm
reference_data = reference_gen()
binning_results = binning(sig, 2, firstbin=True)  # (2x2) pixel image to start
creation_results = model_create(binning_results, reference_data, first_model=True)
fitting_results = model_fit(creation_results, first_fit=True, bounded=True)

# For loop allowing iteration of the algorithm up to user chosen limit.
# "2**scale" defines one of the dimensions of the binned image that is being fitted.
# eg. scale=2 ---> (4x4) or scale=4 ---> (16x16)
for scale in range(2, 7):
    binning_results = binning(sig, int(2**scale))
    
    # model_create now receives fitting_results from previous level
    creation_results = model_create(binning_results, reference_data, first_model=False, fit_results=fitting_results)

    fitting_results = model_fit(creation_results, first_fit=False, bounded=True)

# unpacking results, though this is from creation_results, 
# model_fit will have modified the model such that the instance
# inside creation_results will be fitted
model = creation_results[0]
model.as_signal().plot(cmap='jet')  # Diagnostic (comment out if unnecessary)
disp_grad = model[0].construct_displacement_gradient()

# Strain map generation and plotting
strain_map = disp_grad.get_strain_maps()

# convention to allow viewing of log-scale strain maps without missing pixels
# due to negative values (can comment out if deemed unnecessary)
strain_map *= -1

strain_map.plot(cmap='jet')


AttributeError: 'StrainMap' object has no attribute 'as_signal'